# โค้ดสร้าง CAPPI ในแต่ละความสูง

### อธิบายโค้ด
* โค้ดนี้มีการรัน snr/attenuation correction โดยประมวลผลไฟล์ทั้งหมดก่อน แล้วค่อยนำไปทำ cappi แต่ละมุมที่ต้องการในภายหลัง
* การทำดังนี้ มีผลดีคือ สามารถรันข้อมูลได้เป็นสเต็ป ผลเสียคือ memory เต็ม แล้วประมวลผลได้ไม่เกิน 5 วันของเรดาร์กรมอุตุ ก็มักจะรันต่อไปไม่ได้
* แนวทางแก้ไข ให้ทำ snr/attenuation correction แล้วให้ต่อการสร้าง cappi แต่ละความสูงเลย ประมวลผลให้จบในไฟล์ต่อไฟล์
* นอกจากนั้นโค้ดเดิมนี้ยังมีปัญหา ไม่สามารถอ่านไฟล์ 2018071609 ได้ มักจะหยุดโปรแกรมเลย ดังนั้นควร ignore ไฟล์ที่อ่านไม่ได้ แล้วให้ทำงานต่อ

In [1]:
import os
import numpy as np
import xarray as xr
import rioxarray
import pyart
from datetime import datetime
from dateutil import tz
import copy  # Add this import for the copy module
import warnings
warnings.filterwarnings("ignore")

def utc2local_hr(time_rad):
    from_zone = tz.tzutc()
    to_zone = tz.tzlocal()
    utc = datetime.strptime(time_rad, "%Y%m%d%H")
    utc = utc.replace(tzinfo=from_zone)
    lst = utc.astimezone(to_zone)
    return lst

def snr_atten_cor(radar):
    snr = pyart.retrieve.calculate_snr_from_reflectivity(radar, refl_field='reflectivity', toa=15000.0)
    radar.add_field('signal_to_noise_ratio', snr, replace_existing=True)
    gtfilter = pyart.filters.moment_and_texture_based_gate_filter(radar, phi_field='differential_phase')
    gtfilter.exclude_below('signal_to_noise_ratio', 5)
    gtfilter.exclude_above('signal_to_noise_ratio', 70)
    radar.add_field_like('reflectivity', 'reflectivity_copy', radar.fields['reflectivity']['data'].copy())
    nf = radar.fields['reflectivity_copy']
    nf['data'] = np.ma.masked_where(gtfilter.gate_excluded, nf['data'])
    radar.add_field('filtered_reflectivity', nf, replace_existing=True)

    ncp_values = np.ones((radar.nrays, radar.ngates))
    ncp = pyart.config.get_metadata('normalized_coherent_power')
    ncp['data'] = ncp_values
    radar.add_field('normalized_coherent_power', ncp)

    phidp, kdp = pyart.correct.phase_proc_lp(radar, 0.0, LP_solver='pyglpk', debug=True)
    radar.add_field('proc_dp_phase_shift', phidp)
    radar.add_field('recalculated_diff_phase', kdp)

    spec_at, cor_z = pyart.correct.calculate_attenuation(
        radar,
        0,
        fzl=4500.0,
        refl_field="filtered_reflectivity",
        ncp_field="normalized_coherent_power",
        rhv_field="cross_correlation_ratio",
        phidp_field="proc_dp_phase_shift",
    )
    radar.add_field("specific_attenuation", spec_at)
    radar.add_field("corrected_filtered_reflectivity", cor_z)

    return radar

def gridding_cappi_convert_grid2xarray(radar, height):
    lat_0 = radar.latitude['data'][0]
    lon_0 = radar.longitude['data'][0]

    shape = (11, 241, 241)
    grid = pyart.map.grid_from_radars(
        radar,
        grid_shape=shape,
        grid_limits=((0, 10000), (-240000, 240000), (-240000, 240000)),
        grid_origin=(lat_0, lon_0),
        fields=['corrected_filtered_reflectivity'],
        roi_func='constant', constant_roi=4000.0
    )

    ds = grid.to_xarray()
    grid_data = ds['corrected_filtered_reflectivity'][0, height, :, :]

    return grid_data

def save_geotif(cappi_max_hr, path_outfile):
    cappi_max_hr = cappi_max_hr.to_dataset().squeeze().set_index(x="lon", y="lat")
    cappi_max_hr.rio.set_spatial_dims(x_dim='x', y_dim='y', inplace=True)
    cappi_max_hr.rio.set_crs("EPSG:4326")
    cappi_max_hr.rio.write_crs("EPSG:4326", inplace=True)
    cappi_max_hr.rio.write_transform()
    cappi_max_hr.rio.write_coordinate_system()
    cappi_max_hr.rio.to_raster(path_outfile)
    cappi_max_hr.close()

def main():
    dir_path = 'D:/1Yang/0Geoinformatic_data/0radar_data/phs_2018_events/2Sontihn/'
    #dir_path = 'D:/1Yang/0Geoinformatic_data/0radar_data/phs_2018_events/ztest/'
    path_results = './00Results/0outp_cappi_ppi_hourly/0train_val_models/cappi/'
    #path_results = './00Results/0outp_cappi_ppi_hourly/ztmp/cappi/'

    fn_all = [file for file in os.listdir(dir_path) if file.endswith('.uf.bz2')]
    fn_hr = list(np.unique([file[7:-9] for file in fn_all]))
    #fn_hr = list(np.unique([file[3:-5] for file in fn_all]))

    # Read and preprocess radar data for SNR and attenuation correction
    radar_dict = {}
    print('#1.> ปรับแก้ SNR and attenuation correction...')
    for sub_file_hr in fn_hr:
        files_15min = [file for file in fn_all if sub_file_hr in file]
        
        # Initialize radar_list to store radar data for all files in the current hour
        radar_list = []
        
        print('#---> processing '+sub_file_hr+' ...>>>')
        for file_15min in files_15min:
            radar = pyart.io.read(os.path.join(dir_path, file_15min))
            radar.metadata['file_name'] = file_15min  # Store file name as metadata เก็บไว้เพื่อ print
            radar = snr_atten_cor(radar)
            radar_list.append(radar)

        # Add the list of corrected radar data to the dictionary with the hour as the key
        radar_dict[sub_file_hr] = radar_list

    # Specify the heights you want to generate CAPPI for
    heights = [1, 2, 3, 4]
    print('#2.> สร้างแผนที่ CAPPIs ตามระดับความสูง....')
    for height in heights:
        for sub_file_hr, radar_list in radar_dict.items(): 
            local_time = utc2local_hr(sub_file_hr).strftime('%Y%m%d%H')
            print('---------------------------------------\n')
            print('Find max for .......>>>', sub_file_hr)
            con_xr = []

            for radar in radar_list: 
                #print(f'Doing ...>>> {radar.metadata}') #file_15min อันนี้ไม่เวิร์คนะ ชื่อไม่ตรงกับที่ประมวลผล ต้องลองสัก 2 ชั่วโมงดู
                print('.......>>>', radar.metadata['file_name'])

                # Reuse the corrected radar data for different heights
                radar_copy = copy.deepcopy(radar)  # Use deepcopy instead of copy

                print(f'>> Do gridding CAPPI at height {height} km...')
                grid = gridding_cappi_convert_grid2xarray(radar_copy, height)
                con_xr.append(grid)

            print(f'+ Do find max of CAPPI at {height} km...')
            cappi_con_xr = xr.concat(con_xr, dim='time')
            cappi_max_hr = cappi_con_xr.max(dim='time')

            print(f'+ Do save results for CAPPI at {height} km...')

            # Specify the height for folder creation
            height_folder = height
            folder_name = f'cappi{height_folder}km'

            # Create the folder if it doesn't exist
            folder_path = os.path.join(path_results, folder_name)
            os.makedirs(folder_path, exist_ok=True)

            outp_name = local_time + '.tif'
            save_geotif(cappi_max_hr, os.path.join(folder_path, outp_name))
            print(f'---------------Saved results for CAPPI at {height} km---------------\n')

if __name__ == "__main__":
    main()



## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead


#1.> ปรับแก้ SNR and attenuation correction...
#---> processing 2018071417 ...>>>
Unfolding
Exec time:  1.4870333671569824
Doing  0
Doing  1
Doing  2
Doing  3
Unfolding
Exec time:  1.3758902549743652
Doing  0
Doing  1
Doing  2
Doing  3
Unfolding
Exec time:  1.3828048706054688
Doing  0
Doing  1
Doing  2
Doing  3
Unfolding
Exec time:  1.3824865818023682
Doing  0
Doing  1
Doing  2
Doing  3
#---> processing 2018071418 ...>>>
Unfolding
Exec time:  1.3766295909881592
Doing  0
Doing  1
Doing  2
Doing  3
Unfolding
Exec time:  1.4072303771972656
Doing  0
Doing  1
Doing  2
Doing  3
Unfolding
Exec time:  1.3806633949279785
Doing  0
Doing  1
Doing  2
Doing  3
Unfolding
Exec time:  1.3948743343353271
Doing  0
Doing  1
Doing  2
Doing  3
#---> processing 2018071419 ...>>>
Unfolding
Exec time:  1.370969533920288
Doing  0
Doing  1
Doing  2
Doing  3
Unfolding
Exec time:  1.3821122646331787
Doing  0
Doing  1
Doing  2
Doing  3
Unfolding
Exec time:  1.3618662357330322
Doing  0
Doing  1
Doing  2
Doing  3
Un

TypeError: Unknown or unsupported file format: UNKNOWN